In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
block_size = 500
torch.manual_seed(101)

filename = 'datasets/text/vietnamese_articles.csv'
df = pd.read_csv(filename)
print(df.head(10))
data = df['Contents']
data.dropna(inplace=True)
data = data.values.tolist()

print(data[:10])

special_token = b'\x03'

for idx in range(len(data)):
    if len(data[idx]) > block_size:
        data[idx] = data[idx][:block_size]
    else:
        data[idx] = data[idx] + ' '* (block_size - len(data[idx]))
    data[idx] = data[idx] 

text = ' '.join(data)
text_set = set(text)

stoi = {ch: i for i, ch in enumerate(sorted(text_set))}
itos = {i: ch for i, ch in enumerate(sorted(text_set))}

vocab_size = len(stoi)
encode = lambda x: torch.tensor([stoi[ch] for ch in x], dtype=torch.long)
decode = lambda x: ''.join([itos[i] for i in x])

n_train = int(len(data) * 0.9)
train_data = [encode(data[i]) for i in range(n_train)]
val_data = [encode(data[i]) for i in range(n_train, len(data))]

print(f'vocab_size = {vocab_size}, n_train = {n_train}')



                                                 URL  \
0  https://laodong.vn/bat-dong-san/thong-tin-ngoc...   
1  https://laodong.vn/bat-dong-san/lo-hong-trong-...   
2  https://laodong.vn/bat-dong-san/som-hoan-thien...   
3  https://laodong.vn/bat-dong-san/chi-tiet-ho-so...   
4  https://laodong.vn/bat-dong-san/khoi-tao-khong...   
5  https://laodong.vn/bat-dong-san/can-canh-khu-d...   
6  https://laodong.vn/quy-hoach/tiep-tuc-cuong-ch...   
7  https://laodong.vn/bat-dong-san/nguyen-tac-thi...   
8  https://laodong.vn/bat-dong-san/thanh-hoa-dung...   
9  https://laodong.vn/nha-dep/kinh-nghiem-thue-vi...   

                                               Title  \
0  Thông tin “Ngọc Trinh mua đất ở Bảo Lộc" chỉ l...   
1  Lỗ hổng trong việc thẩm tra năng lực tài chính...   
2  Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...   
3            Chi tiết hồ sơ hoàn công nhà ở năm 2022   
4  Khởi tạo không gian sống đẳng cấp, đón sóng đầ...   
5  Cận cảnh Khu đô thị Thanh Hà bất ngờ bị "cò"

In [3]:
class Head(nn.Module):
    """ one head self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) / (C**0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = F.dropout(wei, p=dropout)
        # perform score aggregation
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_emb, n_emb)

    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        x = F.dropout(x, p=dropout)
        return x

class FeedForward(nn.Module):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4*n_emb),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer Block followed by computation
    """
    def __init__(self, n_emb, n_heads):
        super().__init__()
        self.head_size = n_emb // n_heads
        self.sa = MultiHeadAttention(n_heads, self.head_size)
        self.ff = FeedForward()
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        x = F.dropout(x, p=dropout)
        return x
        
class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb, n_heads) for _ in range(n_layers)])
        self.feed_forward = FeedForward()
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = token_emb + position_emb
        x = self.blocks(x) 
        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self.forward(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
        return idx

batch_size = 32
n_emb = 500
n_layers = 6
n_heads = 5
dropout = 0.2
learning_rate = 3e-4

m = LanguageModel(vocab_size=vocab_size, n_emb=n_emb).to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)


In [10]:


def get_random_subtensors(x, block_size, pad_value=0):
    original_len = x.shape[0]
    pad_value = encode(' ').item()
    
    # Check and pad if necessary
    if block_size >= original_len:
        padded_tensor = torch.full((block_size + 1,), pad_value, dtype=x.dtype)
        padded_tensor[:original_len] = x
        x = padded_tensor
    
    # Get a random start index
    start_index = random.randint(0, len(x) - block_size - 1)
    
    # Extract the subtensors
    subtensor1 = x[start_index:start_index + block_size]
    subtensor2 = x[start_index + 1:start_index + block_size + 1]
    
    return subtensor1, subtensor2


def get_batch(data, block_size, batch_size):
    idx = torch.randint(0, len(data)-1, (batch_size,))
    seq = []
    targets = []
    for i in range(batch_size):
        s, t = get_random_subtensors(data[idx[i].item()], block_size)
        seq.append(s)
        targets.append(t)
    x = torch.stack([seq[i] for i in range(batch_size)])
    y = torch.stack([targets[i] for i in range(batch_size)])
    return x, y

def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

x, y = get_batch(train_data, block_size, 32)
print(x.shape)
print(y.shape)

torch.Size([32, 500])
torch.Size([32, 500])


In [11]:
early_stop = 10
last_val_loss = 1e9
n_epochs = 4000

for steps in range(n_epochs):
    xb, yb = get_batch(train_data, block_size, batch_size)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(m, val_data, block_size, batch_size)
        print('Validation loss:', val_loss)
        if val_loss >= last_val_loss:
            early_stop -= 1
            if early_stop == 0:
                print('Early stop!')
                break
        else:
            early_stop = 10
            last_val_loss = val_loss


Step: 0 Training Loss: 6.092939376831055
Validation loss: 4.841885089874268


In [11]:
print(sum(p.numel() for p in m.parameters() if p.requires_grad))
starting_tokens = 'Thị trường'
len_starting_tokens = len(starting_tokens)
idx = torch.tensor(encode(starting_tokens)).reshape(1, len_starting_tokens).to(device)
print(decode(m.generate(idx, max_new_tokens=2000)[0].tolist()))

20700918


C:\Users\hvutr\AppData\Local\Temp\ipykernel_1448\1578189727.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(encode(starting_tokens)).reshape(1, len_starting_tokens).to(device)


Thị trường xuyên đã clo Bãc lưu nhiều clip 5.  Muốn có cả những thể cảm nhưng đã bị điều trị trên cài thơ. Sự cố tố kết nên axiq đăng lý sở hữu ngựa rộng ngay trên sự nối kè tái chất cấp. Ngay đậm ngắm chày, các thôn làm nhất, có thể cung cất bần. Trước đó, trong khi vào 12 lần chia sẻ xây dựng, điều bị thấy đúc 2 tại đây. Có thời diễn ra 15 năm, mẹ chị làm năng kg quy, Thành phố đã phù hợp với mẫu được lẩm đến nhưng cả tập trung. Mút sáng vật kích ngực với tôi chia sẻ, con để để mong fic hoặc
